In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# --- paths ---
ROOT = Path.cwd().parent            # repo root (notebooks/ -> parent)
PROCESSED_PATH = ROOT / "data" / "processed"

df = pd.read_csv(
    PROCESSED_PATH / "processed_price_data_2023Q1.csv"
)

# Quick look at raw data structure
print("Shape of dataset:", df.shape)
print("\nColumns:")
print(df.columns)
df.head()   # Display first 5 rows


# --- Clean Price column ---
# Remove 'Rs. ' and commas, convert to numeric
df['Price2'] = (
    df['Price']
    .str.replace('Rs. ', '', regex=False)
    .str.replace(',', '')
    .astype(float)
)

# Show original vs cleaned price
df[['Price', 'Price2']].head()

Shape of dataset: (500, 11)

Columns:
Index(['Price', 'Validity', 'Description', 'Tags', 'Circle', 'Time', 'Price2',
       'Days2', 'Tag1', 'Tag2', 'clean_price_per_day'],
      dtype='object')


,Price,Price2
0,Rs. 58,58.0
1,Rs. 2998,2998.0
2,Rs. 2997,2997.0
3,Rs. 899,899.0
4,Rs. 756,756.0


In [4]:
# --- Extract validity days ---
def extract_days(val):
    """
    Extract number of days from the 'Validity' column.
    Example: '28 days' -> 28
    """
    if pd.isna(val) or val == 'N.A.':
        return np.nan
    val = str(val).lower()
    if 'day' in val:
        try:
            return int(val.split()[0])
        except:
            return np.nan
    return np.nan

df['Days2'] = df['Validity'].apply(extract_days)

# Inspect extracted days
df[['Validity', 'Days2']].head()


# --- Split company tags ---
# Extract first word as company (Tag1), rest as Tag2
df[['Tag1', 'Tag2']] = df['Tags'].str.extract(r'([^ ]+)\s+(.*)')

# Inspect tag split
df[['Tags', 'Tag1', 'Tag2']].head()


# --- Calculate clean price per day ---
df['clean_price_per_day'] = df['Price2'] / df['Days2']

# Inspect calculation
df[['Price2', 'Days2', 'clean_price_per_day']].head()


# --- Compute average price per day by circle and company ---
df_price_long = df[['Circle', 'Tag1', 'clean_price_per_day']].copy()
df_price_long.columns = ['circle', 'company', 'avg_price']

df_price_long = (
    df_price_long
    .groupby(['circle', 'company'], as_index=False)['avg_price']
    .mean()
)

# Show aggregated results
df_price_long.head()


# --- Save final dataset ---
df_price_long.to_excel(
    PROCESSED_PATH / "price_final_2023Q1.xlsx",
    index=False
)

print("\nFinal dataset saved to:", PROCESSED_PATH / "price_final_2023Q1.xlsx")
print("Final shape:", df_price_long.shape)


Final dataset saved to: /home/jupyter-xueyin/GogoLeoTaki/data/processed/price_final_2023Q1.xlsx
Final shape: (36, 3)
